# Naive Bayes Classifier: Spam Detection

## Defining the question

### a.) Specifying the question

#### * Is the email sent a spam or not a spam ? 

In [103]:
# Importing all the necessary libraries for my analysis
# EDA and Cleaning
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Feature Engineering ,Preprocessing and Modeling
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder,Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.naive_bayes import GaussianNB
# Function to diable warnings
import warnings
warnings.filterwarnings("ignore")

In [104]:
# Reading the dataset
spam_df=pd.read_csv('spambase\spambase.data.csv',header=None)
spam_df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [105]:
# Checking the structure of our dataset
spam_df.shape

(4601, 58)

#### Cleaning proceedures

In [106]:
# Checking for missing values in the dataset.
spam_df.isnull().any().sum()
# There are no missing values in the set

0

In [107]:
# Checking for the datatypes
spam_df.dtypes.unique()
# There are numeric datastypes in the set

array([dtype('float64'), dtype('int64')], dtype=object)

In [108]:
# Data Preparation
feat=spam_df.drop(columns=[57],axis=1)
X=feat.values
label=spam_df[57]
Y=label.values

In [109]:
# Checking for normality of my features 
from scipy.stats import shapiro
# Extracting test statistic and the p_value to compare it to a threshhold
stat,p_val=shapiro(X)
print(stat,p_val)
thresh=0.05
if p_val>thresh:
    print('The feature distribution is Gaussian')
else:
    print('The distribution of features is not Gaussian')

0.03710019588470459 0.0
The distribution of features is not Gaussian


#### For the reason above normalizing will be one of the preprocessing techniquest to be performed bef0re fitting our set

### Feature Engineering and Preprocessing proceedures

In [110]:
# Splitting our set into training and testing sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

In [111]:
# Normalizing of our features since the distribution wasn't Gaussian making them range from 0 t0 1
norm=Normalizer(norm='l2')
X_train=norm.fit_transform(X_train)
X_test=norm.transform(X_test)

## Feature reduction

In [112]:
# Reducing columns from 58 to 10
lda=LinearDiscriminantAnalysis(n_components=1,solver='eigen',)
# reducing both the training and the testing set
X_train=lda.fit_transform(X_train,Y_train)
X_test=lda.transform(X_test)

In [113]:
# Instantiating our Classifier
gauss=GaussianNB()
# Fitting our model
gauss.fit(X_train,Y_train)
# Predicting the mail status whether spam or not
Y_pred=gauss.predict(X_test)
# Evaluating this model
ac_nb=accuracy_score(Y_test,Y_pred)
cm_nb=confusion_matrix(Y_test,Y_pred)
cr_nb=classification_report(Y_test,Y_pred)
print(ac_nb,'\n',cm_nb,'\n',cr_nb) 

0.7709011943539631 
 [[473  85]
 [126 237]] 
               precision    recall  f1-score   support

           0       0.79      0.85      0.82       558
           1       0.74      0.65      0.69       363

    accuracy                           0.77       921
   macro avg       0.76      0.75      0.75       921
weighted avg       0.77      0.77      0.77       921



#### The model scored a 77 percent accuracy which a alot of emails being missclassified and regarded as spams. This is below our expectation as the aim is to have a model that correctly classifies emails as spam and an email with minimal error.

### Optimizing our model